In [1]:
import cv2
import numpy as np
import random
import math
import pandas as pd
from copy import deepcopy
import matplotlib.pyplot as plt
import time
import json

source_image = cv2.imread("painting.png")
width = source_image.shape[1]
height = source_image.shape[0]

max_radius = 45
num_generations = 10000

num_inds = 20
num_genes = 50
tm_size = 5
frac_elites = 0.2
frac_parents = 0.6
mutation_prob = 0.2
mutation_type = "guided"
default_list = [num_inds, num_genes, tm_size, frac_elites, frac_parents, mutation_prob, mutation_type]

num_inds_list = [5, 10, 20, 40, 60]
num_genes_list = [15, 30, 50, 80, 120]
tm_size_list = [2, 5, 8, 16]
frac_elites_list = [0.04, 0.2, 0.35]
frac_parents_list = [0.15, 0.3, 0.6, 0.75]
mutation_prob_list = [0.1, 0.2, 0.4, 0.75]
mutation_type_list = ["unguided", "guided"]
parameters = [num_inds_list,num_genes_list,tm_size_list,frac_elites_list,frac_parents_list,mutation_prob_list,mutation_type_list]
names = ["num_inds","num_genes","tm_size","frac_elites","frac_parents","mutation_prob","mutation_type"]

In [2]:
class Gene:
    def __init__(self, x, y, radius, R, G, B, A):
        self.x = x
        self.y = y
        self.radius = radius
        self.R = R
        self.G = G
        self.B = B
        self.A = A

class Individual:
    def __init__(self, num_genes):
        self.ID = -1  # ID of the individual
        self.chromosome = []  # List of genes representing circles
        self.fitness = -99999999999  # Fitness value of the individual
        for _ in range(num_genes):
            outside = True
            while outside:
                gene = Gene(
                    x=random.randint(0 - max_radius, width + max_radius),  # Random x-coordinate
                    y=random.randint(0 - max_radius, height + max_radius),  # Random y-coordinate
                    radius=random.randint(1, max_radius),  # Random radius
                    R=random.randint(0, 255),  # Random red value
                    G=random.randint(0, 255),  # Random green value
                    B=random.randint(0, 255),  # Random blue value
                    A=random.uniform(0, 1),  # Random alpha value
                )
                outside = is_outside(gene.x, gene.y, gene.radius)
            self.chromosome.append(gene)
        self.chromosome.sort(key=lambda gene: gene.radius, reverse=True)

def Population(num_individuals, num_genes):
    individuals = []  # List of individuals
    for i in range(num_individuals):
        individual = Individual(num_genes)
        individual.ID = i
        individuals.append(individual)
    individuals.sort(key=lambda ind: ind.fitness, reverse=True)
    return individuals

def evaluate_individual(individual, source_image):
    individual.chromosome.sort(key=lambda gene: gene.radius, reverse=True)
    #image = np.zeros_like(source_image)  
    image = np.zeros_like(source_image, dtype=np.uint8) # Initialize image with zeros
    image.fill(255)
    for gene in individual.chromosome:
        overlay = deepcopy(image)  # Create a copy of the image
        
        # Extract gene attributes
        x = gene.x
        y = gene.y
        radius = gene.radius
        R = gene.R
        G = gene.G
        B = gene.B
        A = gene.A
        
        # Draw the circle on the overlay
        cv2.circle(overlay, (x, y), radius, (B, G, R), -1)
        
        # Apply alpha blending to overlay the circle on the image
        image = cv2.addWeighted(overlay, A, image, 1 - A, 0)
    

    # Calculate fitness value by comparing the generated image with the source image
    srcimg_img=np.subtract(np.array(source_image, dtype=np.int64), np.array(image, dtype=np.int64))
    fitness = np.sum(-1*np.power(srcimg_img, 2))

    # Update the individual's fitness attribute
    individual.fitness = fitness

def is_outside(x, y, radius):
    outside = True
    # Check if the circle is outside the image
    # 1. inside, middle, middle
    if (x >= 0 and x <= width) and (y >= 0 and y <= height):
        outside = False
    # 2. outside, left, middle
    elif (x < 0) and (y > 0 and y < height):
        if (x + radius<0):
            outside = True
    # 3. outside, right, middle
    elif (x > width) and (y > 0 and y < height):
        if (x - radius>width):
            outside = True
    # 4. outside, bottom, middle
    elif (y < 0) and (x > 0 and x < width):
        if (y + radius<0):
            outside = True
    # 5. outside, top, middle
    elif (y > height) and (x > 0 and x < width):
        if (y - radius>height):
            outside = True
    # 6. outside, left, bottom
    elif (x < 0) and (y < 0):
        if (radius**2 < (x - 0)**2 + (y - 0)**2):
            outside = True
    # 7. outside, left, top
    elif (x < 0) and (y > height):
        if (radius**2 < (x - 0)**2 + (y - height)**2):
            outside = True
    # 8. outside, right, bottom
    elif (x > width) and (y < 0):
        if (radius**2 < (x - width)**2 + (y - 0)**2):
            outside = True
    # 9. outside, right, top
    elif (x > width) and (y > height):
        if (radius**2 < (x - width)**2 + (y - width)**2):
            outside = True
    else:
        outside = False
    return outside

def selection(population, elites_IDs, num_parents):
    selected_parents = []
    selected_ids = []
    parent_candidates = deepcopy([ind for ind in population if ind.ID not in elites_IDs])
    parent_candidates_ids = [ind.ID for ind in parent_candidates]
    for _ in range(num_parents):
        best_cand = random.choice(parent_candidates)
        best_cand_id = best_cand.ID
        for i in range(tm_size):
            cand = random.choice(parent_candidates)
            if cand.fitness > best_cand.fitness:
                best_cand_id = cand.ID
                best_cand = cand
        selected_ids.append(best_cand_id)
        parent_candidates_ids.remove(best_cand_id)
        parent_candidates.remove(best_cand)
    selected_parents = deepcopy([ind for ind in population if ind.ID in selected_ids])
    return selected_parents, selected_ids

def selection_old(population, elites_IDs, num_parents):
    selected_parents = []
    selected_ids = []
    parent_candidates = deepcopy([ind for ind in population if ind.ID not in elites_IDs])
    for _ in range(num_parents):
        while True:
            tournament = random.sample(parent_candidates, min(tm_size, num_parents, len(parent_candidates)))
            tournament.sort(key=lambda ind: ind.fitness, reverse=True)
            winner = tournament[0]
            if winner.ID not in selected_ids:
                selected_parents.append(deepcopy(winner))
                selected_ids.append(winner.ID)
                parent_candidates.remove(winner)
                break
    selected_parents = deepcopy(selected_parents)
    return selected_parents, selected_ids

def crossover(parent1, parent2):
    chromosome_length = len(parent1.chromosome)
    cand1 = Individual(chromosome_length)
    cand2 = Individual(chromosome_length)
    # Perform crossover
    for gene in range(chromosome_length):
        coinflip = random.randint(0, 1)
        if coinflip == 0:
            cand1.chromosome[gene] = deepcopy(parent1.chromosome[gene])
            cand2.chromosome[gene] = deepcopy(parent2.chromosome[gene])

        else:
            cand1.chromosome[gene] = deepcopy(parent2.chromosome[gene])
            cand2.chromosome[gene] = deepcopy(parent1.chromosome[gene])

    evaluate_individual(cand1, source_image)
    evaluate_individual(cand2, source_image)
    inds = [cand1, cand2, parent1, parent2]
    inds.sort(key=lambda ind: ind.fitness, reverse=True)
    child1 = deepcopy(inds[0])
    child2 = deepcopy(inds[1])
    child1.ID = parent1.ID
    child2.ID = parent2.ID
    return child1, child2

def mutate(individual):
    while True:
        prev_fitness = individual.fitness
        temp_ind = deepcopy(individual)
        for gene in range(len(temp_ind.chromosome)):
            if random.random() < mutation_prob:
                if mutation_type == "unguided":
                    mutate_unguided(temp_ind.chromosome[gene])
                elif mutation_type == "guided":
                    mutate_guided(temp_ind.chromosome[gene])
        evaluate_individual(temp_ind, source_image)
        if temp_ind.fitness > prev_fitness:
            individual.chromosome = deepcopy(temp_ind.chromosome)
            individual.fitness = temp_ind.fitness
            break
        else:
            break
    return individual

def mutate_unguided(gene):
        outside = True
        while outside:
            gene.x = random.randint(0 - max_radius, width + max_radius)
            gene.y = random.randint(0 - max_radius, height + max_radius)
            gene.radius = random.randint(1, max_radius)
            outside = is_outside(gene.x, gene.y, gene.radius)
        gene.R = random.randint(0, 255)
        gene.G = random.randint(0, 255)
        gene.B = random.randint(0, 255)
        gene.A = random.uniform(0, 1)

def mutate_guided(gene):
    # Mutate the gene attributes without exceeding the boundaries
    x = gene.x
    y = gene.y
    radius = gene.radius
    R = gene.R
    G = gene.G
    B = gene.B
    A = gene.A
    temp_x = x
    temp_y = y
    temp_radius = radius
    outside = True
    while outside:
        temp_x = x + random.randint(-width // 4, width // 4)
        if (temp_x < x):
            temp_x = max(temp_x, 0 - max_radius)
        else:
            temp_x = min(temp_x, width + max_radius)

        temp_y = y + random.randint(-height // 4, height // 4)
        if (temp_y < y):
            temp_y = max(temp_y, 0 - max_radius)
        else:
            temp_y = min(temp_y, height + max_radius)

        temp_radius = radius + random.randint(-10, 10)
        if (temp_radius < 0):
            temp_radius = 1
        else:
            temp_radius = min(temp_radius, max_radius)
        outside = is_outside(temp_x, temp_y, temp_radius)
    
    gene.x = temp_x
    gene.y = temp_y
    gene.radius = temp_radius

    R = gene.R + random.randint(-64, 64)
    if (R >= 0 and R <= 255):
        gene.R = R
    elif (R < 0):
        gene.R = 0
    elif (R > 255):
        gene.R = 255

    G = gene.G + random.randint(-64, 64)
    if (G >= 0 and G <= 255):
        gene.G = G
    elif (G < 0):
        gene.G = 0
    elif (G > 255):
        gene.G = 255
    
    B = gene.B + random.randint(-64, 64)
    if (B >= 0 and B <= 255):
        gene.B = B
    elif (B < 0):
        gene.B = 0
    elif (B > 255):
        gene.B = 255

    A = gene.A + random.uniform(-0.25, 0.25)
    if (A >= 0 and A <= 1):
        gene.A = A
    elif (A < 0):
        gene.A = 0
    elif (A > 1):
        gene.A = 1

def draw_circle(individual, name, value, generation):
    individual.chromosome.sort(key=lambda gene: gene.radius, reverse=True)
    image = np.zeros_like(source_image, dtype=np.uint8) # Initialize image with zeros
    image.fill(255)
    for gene in individual.chromosome:
        overlay = deepcopy(image)
        x = gene.x
        y = gene.y
        radius = gene.radius
        color = (gene.B, gene.G, gene.R)
        A = gene.A
        thickness = -1  # Filled circle

        cv2.circle(overlay, (x, y), radius, color, thickness)
        cv2.addWeighted(overlay, A, image, 1 - A, 0, image)
    cv2.imwrite(f"C:/Users/erkan/Desktop/EE/e2022_2/EE449/2023/HW2/Code/{name}/img_{name}_{value}_gen{generation}.png", image)

def draw_fig(fitness_list, name, value):
    part1 = int(num_generations/10)
    part2 = num_generations
    parts = [part1, part2]
    print(len(fitness_list[0:10]))
    print(len(fitness_list[10:]))
    for part in parts:
        if part == part1:
            generations = range(1, part1+1)
            plt.plot(generations, fitness_list[0:part1])
            plt.title(f'Fitness vs Generation {name}={value} {1}-{part1}')
        else:
            generations = range(part1+1, part2+1)
            plt.plot(generations, fitness_list[part1:])
            plt.title(f'Fitness vs Generation {name}={value} {part1}-{part2}')
        plt.xlabel('Generation')
        plt.ylabel('Fitness')
        plt.grid()
        plt.savefig(f"C:/Users/erkan/Desktop/EE/e2022_2/EE449/2023/HW2/Code/{name}/fig_{name}_{value}_{part}.png")
        plt.close()

def genetic_algorithm(name, item):
    # Step 6.1: Initialize the population with random individuals
    population = Population(num_inds, num_genes)
    fitness_list = []
    population_sorted = []
    # Step 6.2: Iterate over the specified number of generations
    for generation in range(num_generations):
        if (generation+1) % (num_generations/100) == 0:
            print(f"Generation {generation+1}/{num_generations}, Best Fitness: {population[0].fitness}, Parameters: num_inds={num_inds}, num_genes={num_genes}, tm_size={tm_size}, frac_elites={frac_elites}, frac_parents={frac_parents}, mutation_prob={mutation_prob}, mutation_type={mutation_type}")
            print(f"Time: parent={round(parent_time,3)}, crossover={round(crossover_time,3)}, mutation={round(mutation_time,3)}")
        if (generation+1) % (num_generations/10) == 0 or (generation == 0):
            draw_circle(population[0], name, item, generation+1)

        # Step 6.3: Evaluate all individuals in the population
        for individual in population:
            evaluate_individual(individual, source_image)
        population.sort(key=lambda ind: ind.fitness, reverse=True)

        fitness_list.append(population[0].fitness)

        # Step 6.4: Select elites to directly pass to the next generation
        num_elites = int(frac_elites * num_inds)
        elites = deepcopy(population[:num_elites])
        elites_IDs = [ind.ID for ind in elites]

        # Step 6.5: Perform tournament selection to select parents for crossover
        num_parents = int(frac_parents * num_inds)
        if num_parents % 2 != 0:
            num_parents += 1
        parent_start = time.time()
        parents, parents_IDs = selection(population, elites_IDs, num_parents)
        parent_end = time.time()
        parent_time = parent_end - parent_start
 
        nonparents_IDs = [ind.ID for ind in population if ind.ID not in parents_IDs]
        nonparents_IDs = [npID for npID in nonparents_IDs if npID not in elites_IDs]
        nonparents = deepcopy([ind for ind in population if ind.ID in nonparents_IDs])

        # Step 6.6: Apply crossover to create new individuals
        offspring = []
        crossover_start = time.time()
        for i in range(0, num_parents, 2):
            # Perform crossover on adjacent parents
            parent1 = parents.pop(random.randint(0,len(parents)-1))
            parent2 = parents.pop(random.randint(0,len(parents)-1))
            #parent1 = parents.pop(0)
            #parent2 = parents.pop(0)
            # parent2 = parents[i+1]
            child1, child2 = crossover(parent1, parent2)
            offspring.extend([child1, child2])
        crossover_end = time.time()
        crossover_time = crossover_end - crossover_start
        # Step 6.7: Perform mutation on some individuals
        mutation_candidates = deepcopy(offspring + nonparents)
        mutation_results = []
        mutation_start = time.time()
        for individual in mutation_candidates:
            individual = mutate(individual)
            mutation_results.append(individual)
        mutation_end = time.time()
        mutation_time = mutation_end - mutation_start
        # Step 6.7: Update the population with elites, mutation results
        population = deepcopy(elites + mutation_results)

        # Sort the final population based on fitness values in descending order
        population.sort(key=lambda ind: ind.fitness, reverse=True)
        # Print population IDs
        
        #print(f"Generation {generation+1}/{num_generations}, times: parent={parent_time}, crossover={crossover_time}, mutation={mutation_time}")
    # Return the final population
    return population, fitness_list

In [3]:
'''
print(f"Running for default parameters")
population, fitness_list = genetic_algorithm("default_parameters", "default")
best_individual = population[0]
draw_fig(fitness_list, "default_parameters", "default")
'''
for param, name in zip(parameters,names):
    num_inds, num_genes, tm_size, frac_elites, frac_parents, mutation_prob, mutation_type = default_list
    if name != "num_inds":
        continue
    for item in param:
        if name == "num_inds":
            num_inds = item
        elif name == "num_genes":
            num_genes = item
        elif name == "tm_size":
            tm_size = item
        elif name == "frac_elites":
            frac_elites = item
        elif name == "frac_parents":
            frac_parents = item
        elif name == "mutation_prob":
            mutation_prob = item
        elif name == "mutation_type":
            mutation_type = item
        print(f"Running for num_inds={num_inds}, num_genes={num_genes}, tm_size={tm_size}, frac_elites={frac_elites}, frac_parents={frac_parents}, mutation_prob={mutation_prob}, mutation_type={mutation_type}")
        population, fitness_list = genetic_algorithm(name, item)

        # Find the best individual from the final population
        best_individual = population[0]

        # Plot the fitness graph
        draw_fig(fitness_list, name, item)

# Done
print("Done")

Running for num_inds=5, num_genes=50, tm_size=5, frac_elites=0.2, frac_parents=0.6, mutation_prob=0.2, mutation_type=guided
Generation 100/10000, Best Fitness: -348399224, Parameters: num_inds=5, num_genes=50, tm_size=5, frac_elites=0.2, frac_parents=0.6, mutation_prob=0.2, mutation_type=guided
Time: parent=0.003, crossover=0.013, mutation=0.012
Generation 200/10000, Best Fitness: -300674352, Parameters: num_inds=5, num_genes=50, tm_size=5, frac_elites=0.2, frac_parents=0.6, mutation_prob=0.2, mutation_type=guided
Time: parent=0.003, crossover=0.015, mutation=0.01
Generation 300/10000, Best Fitness: -266590099, Parameters: num_inds=5, num_genes=50, tm_size=5, frac_elites=0.2, frac_parents=0.6, mutation_prob=0.2, mutation_type=guided
Time: parent=0.003, crossover=0.015, mutation=0.011
Generation 400/10000, Best Fitness: -255233712, Parameters: num_inds=5, num_genes=50, tm_size=5, frac_elites=0.2, frac_parents=0.6, mutation_prob=0.2, mutation_type=guided
Time: parent=0.003, crossover=0.0